In [1]:
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
from unet import UNet  # 假设 UNet 是 unet.py 中定义的一个类
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import os
from __future__ import print_function, division
import os
from PIL import Image
import torch
import torch.utils.data
import torchvision.transforms as transforms
from torchvision.transforms import functional as F
from skimage import io
from torch.utils.data import Dataset

import numpy as np
import dataset



In [3]:
image_transform = transforms.Compose([
    transforms.ToPILImage(),  # 如果图像不是PIL图像则需要转换
    transforms.Resize((256, 1024)),  # 调整图像大小
    transforms.ToTensor(),  # 将图像转换为torch.Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 标准化
])

# 掩膜的转换操作，掩膜不需要标准化
mask_transform = transforms.Compose([
    transforms.ToPILImage(),  # 如果掩膜不是PIL图像则需要转换
    transforms.Resize((256, 1024), interpolation=transforms.InterpolationMode.NEAREST),  # 调整大小，使用最近邻插值
    transforms.ToTensor()  # 将掩膜转换为torch.Tensor
])

In [4]:
# image_transform = transforms.Compose([
#      transforms.Resize((256, 1024)),
#      transforms.ToTensor()
# ])

# mask_transform = transforms.Compose([
#      transforms.Resize((256, 1024)),
#      transforms.ToTensor()  # Assuming 'x' is a numpy array
# ])
trainset = dataset.KITTIDataset('tinySet/img', 'tinySet/mask', transform=image_transform, mask_transform=mask_transform)
dataloader = DataLoader(trainset, batch_size=4, shuffle=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)



In [5]:
for i, (images, masks) in enumerate(dataloader):
     print(masks.shape)
     print(images.shape)
     break

torch.Size([4, 1, 256, 1024])
torch.Size([4, 3, 256, 1024])


In [ ]:
def train(model, loader, optimizer, criterion, n_epochs):
    for epoch in range(n_epochs):
        model.train()
        for i, (images, masks) in enumerate(loader):
            images = images.to(device)
            masks = masks.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()
        # print(f'Epoch {epoch+1}, Loss: {loss.item()}, IoU: {iou_score}')
train(model, dataloader, optimizer, criterion, 10)
# 保存整个模型
torch.save(model, 'model_complete.pth')

In [8]:
import os
import torch
from torchvision.utils import save_image
import numpy as np

def predict_and_save(model, loader, save_dir):
    model.eval()
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    with torch.no_grad():
        for images, filenames in loader:
            # 假设你的模型和数据都在同一设备上
            outputs = model(images)
            outputs = torch.sigmoid(outputs)  # 转换为概率

            # outputs shape is expected to be (batch_size, 1, H, W) after sigmoid
            outputs = outputs.squeeze(1)  # 去除通道维度，变为 (batch_size, H, W)

            for i, filename in enumerate(filenames):
                save_path = os.path.join(save_dir, filename)
                output = outputs[i].cpu().numpy()  # 转换为numpy数组
                output = (output > 0.5).astype(np.uint8)  # 应用阈值并转换为二进制图像
                save_image(torch.tensor(output), save_path)  # 保存图像

# 初始化和调用你的模型和数据加载器
test_dataset = dataset.TestDataset('tinySet/img', transform=image_transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
save_masks_dir = 'predicted_masks'
predict_and_save(model, test_loader, save_masks_dir)


tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])
tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])
tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])


KeyboardInterrupt: 